<a href="https://colab.research.google.com/github/Elshan777/transfomers-from-scratch/blob/master/Transformers%20from%20Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install the Data

In [1]:
%pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 30.4 MB/s 
     |████████████████████████████████| 212 kB 73.4 MB/s 
     |████████████████████████████████| 115 kB 64.2 MB/s 
     |████████████████████████████████| 120 kB 60.7 MB/s 
     |████████████████████████████████| 127 kB 62.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [73]:
import datasets

In [74]:
all_ds = datasets.list_datasets()
len(all_ds)

8773

In [75]:
dataset = datasets.load_dataset('oscar', 'unshuffled_deduplicated_az')

Generating train split:   0%|          | 0/626796 [00:00<?, ? examples/s]

Dataset oscar downloaded and prepared to /root/.cache/huggingface/datasets/oscar/unshuffled_deduplicated_az/1.0.0/84838bd49d2295f62008383b05620571535451d84545037bb94d6f3501651df2. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [76]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'text'],
        num_rows: 626796
    })
})

In [77]:
dataset['train'][0]

{'id': 0,
 'text': 'AZTV-Artıq 7 ildir ki, Abşeron rayonu dotasiya almadan bütün xərclərini yerli daxilolmalar hesabına maliyyələşdirir.\nDünən, 10:49 | Azərbaycanda Konstitusiya GünüdürDünən, 10:46 | Yapon tədqiqatçılar Parkinson xəstəliyinin müalicəsi istiqamətində mühüm uğur qazanıblarDünən, 10:44 | Azərbaycan İstanbul Beynəlxalq Kitab Sərgisində təmsil olunur10-11-2018, 19:56 | İradə Gülməmmədova şahmat üzrə üçqat Avropa çempionu ilə görüşdü- FOTO10-11-2018, 11:18 | Süni intellektli ilk bankomat istifadəyə verilib10-11-2018, 11:16 | Çikaqoda Dövlət Bayrağı Günü münasibətilə tədbir keçirilib10-11-2018, 11:12 | Azərbaycan ilə Türkiyə arasında QHT-lərlə bağlı memorandum imzalanıb9-11-2018, 19:05 | Abşeronun yeniyetmə cüdoçuları zona mərhələsinin qalibi adını qazanıblar9-11-2018, 16:44 | Ceyranbatan 1 nömrəli məktəbdə Bayraq Günü tədbiri-Video9-11-2018, 15:25 | Xırdalanın 23 saylı bağçasında Dövlət Bayrağı Günü qeyd olunub (Foto+Video)9-11-2018, 15:09 | Xırdalan şəhər 3 saylı uşaq bağç

In [78]:
from tqdm.auto import tqdm  # for our loading bar

text_data = []
file_count = 0

for sample in tqdm(dataset['train']):
    # remove newline characters from each sample as we need to use exclusively as seperators
    sample = sample['text'].replace('\n', '')
    text_data.append(sample)
    if len(text_data) == 5_000:
        # once we hit the 5K mark, save to file
        with open(f'text_{file_count}.txt', 'w', encoding='utf-8') as fp:
            fp.write('\n'.join(text_data))
        text_data = []
        file_count += 1
# after saving in 5K chunks, we will have ~3808 leftover samples, we save those now too
with open(f'text_{file_count}.txt', 'w', encoding='utf-8') as fp:
    fp.write('\n'.join(text_data))


  0%|          | 0/626796 [00:00<?, ?it/s]

## Build a Custom Transformer Tokenizer

In [79]:
from pathlib import Path
import os

In [80]:
paths = [str(x) for x in Path('').glob('**/*.txt')]
paths

['text_95.txt',
 'text_97.txt',
 'text_80.txt',
 'text_33.txt',
 'text_74.txt',
 'text_41.txt',
 'text_28.txt',
 'text_63.txt',
 'text_108.txt',
 'text_44.txt',
 'text_23.txt',
 'text_18.txt',
 'text_69.txt',
 'text_47.txt',
 'text_109.txt',
 'text_26.txt',
 'text_114.txt',
 'text_103.txt',
 'text_117.txt',
 'text_77.txt',
 'text_90.txt',
 'text_55.txt',
 'text_72.txt',
 'text_27.txt',
 'text_36.txt',
 'text_29.txt',
 'text_60.txt',
 'text_81.txt',
 'text_19.txt',
 'text_24.txt',
 'text_89.txt',
 'text_116.txt',
 'text_17.txt',
 'text_94.txt',
 'text_57.txt',
 'text_99.txt',
 'text_11.txt',
 'text_21.txt',
 'text_45.txt',
 'text_56.txt',
 'text_43.txt',
 'text_2.txt',
 'text_102.txt',
 'text_101.txt',
 'text_15.txt',
 'text_70.txt',
 'text_61.txt',
 'text_96.txt',
 'text_4.txt',
 'text_14.txt',
 'text_88.txt',
 'text_65.txt',
 'text_22.txt',
 'text_115.txt',
 'text_75.txt',
 'text_62.txt',
 'text_76.txt',
 'text_20.txt',
 'text_32.txt',
 'text_31.txt',
 'text_25.txt',
 'text_121.txt',


In [10]:
%pip install tokenizers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.6 MB 36.3 MB/s 


In [81]:
from tokenizers import ByteLevelBPETokenizer

In [82]:
tokenizer = ByteLevelBPETokenizer()

In [83]:
tokenizer.train(files=paths, vocab_size=30_522, min_frequency=2,
                    special_tokens=['<s>', '<pad>', '</s>', '<unk>', '<mask>'])

In [84]:
os.mkdir('bert_dayi')

tokenizer.save_model('bert_dayi')

['bert_dayi/vocab.json', 'bert_dayi/merges.txt']

#### Using the tokenizer

In [15]:
%pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 18.3 MB/s 


In [85]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained('bert_dayi', max_len=512) # load the tokenizer

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'RobertaTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'RobertaTokenizerFast'.


In [69]:
lorem_ipsum = (
)

In [70]:
tokenizer(lorem_ipsum, max_length=512, padding='max_length', truncation=True)

{'input_ids': [0, 3586, 652, 1600, 460, 1787, 16, 2617, 3713, 3087, 16, 397, 3701, 13753, 3726, 16098, 329, 2218, 290, 1913, 1546, 1649, 18, 1375, 411, 319, 10177, 1930, 16, 631, 13321, 23665, 2437, 6331, 9088, 690, 329, 20863, 349, 506, 7541, 10802, 18, 15643, 379, 73, 2919, 2649, 1600, 285, 2067, 285, 1603, 1255, 360, 17170, 1913, 2513, 2073, 1088, 2523, 18, 15441, 28800, 908, 24535, 30304, 312, 20855, 16, 337, 285, 2526, 365, 2572, 3044, 17796, 580, 461, 297, 3561, 18, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

## Building MLM Training Input Pipeline

In [71]:
with open('text_0.txt', 'r', encoding='utf-8') as fp:
    lines = fp.read().split('\n')

In [72]:
batch = tokenizer(lines, max_length=512, padding='max_length', truncation=True)
len(batch)

2

In [21]:
import torch

def mlm(tensor):
    rand = torch.rand(tensor.shape)
    mask_arr = rand < 0.15 * (tensor > 2)
    for i in range(tensor.shape[0]):
        selection = torch.flatten(mask_arr[i].nonzero())
        tensor[i, selection] = 4
    return tensor

In [22]:
from pathlib import Path

paths = [str(x) for x in Path('').glob('*.txt')]
paths[:5]

['text_2.txt', 'text_1.txt', 'text_3.txt', 'text_0.txt']

In [23]:
from tqdm.auto import tqdm

input_ids = []
mask = []
labels = []

for path in tqdm(paths):
    with open(path, 'r', encoding='utf-8') as fp:
        lines = fp.read().split('\n')
    sample = tokenizer(lines, max_length=512, padding='max_length', truncation=True, return_tensors='pt')
    # input_ids.append(batch['input_ids'])
    # mask.append(batch['attention_mask'])
    labels.append(sample.input_ids)
    mask.append(sample.attention_mask)
    input_ids.append(mlm(sample.input_ids.detach().clone()))

  0%|          | 0/4 [00:00<?, ?it/s]

In [24]:
input_ids = torch.cat(input_ids)
mask = torch.cat(mask)
labels = torch.cat(labels)

In [25]:
input_ids[0][:10]

tensor([   0,   49,  431, 1146,  693,    4,  290, 5720, 6298, 1641])

In [26]:
encodings = {
    'input_ids': input_ids,
    'attention_mask': mask,
    'labels': labels
}

In [27]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encoding):
        self.encoding = encoding
    def __len__(self):
        return self.encoding['input_ids'].shape[0]
    def __getitem__(self, idx):
        return {key: tensor[idx] for key, tensor in self.encoding.items()}

In [28]:
dataset = Dataset(encodings)

In [46]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=10, shuffle=True)

## Training and Testing Latin BERT

In [47]:
from transformers import RobertaConfig
config =  RobertaConfig(
    vocab_size=tokenizer.vocab_size,
    max_position_embeddings=514,
    hidden_size=768,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

In [48]:
from transformers import RobertaForMaskedLM

In [49]:
model = RobertaForMaskedLM(config)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
model.to(device)

In [51]:
from transformers import AdamW

# activate training mode
model.train()
# initialize optimizer
optim = AdamW(model.parameters(), lr=1e-4)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [52]:
from tqdm.auto import tqdm

In [53]:
epochs = 2

In [54]:
torch.cuda.empty_cache()

In [56]:
for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(dataloader, leave=True)
    for batch in loop:
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device) 
        outputs = model(input_ids, attention_mask=mask, labels=labels)

        loss = outputs.loss
        loss.backward()
        optim.step()

        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

  0%|          | 0/1881 [00:00<?, ?it/s]

  0%|          | 0/1881 [00:00<?, ?it/s]

In [62]:
from transformers import pipeline


In [67]:
fill = pipeline('fill-mask', model='bertius', tokenizer='bertius')

OSError: ignored